# Analyze correlator results
Dec 10, 2020

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys
import glob

In [2]:
%matplotlib widget

In [3]:
from ipywidgets import *

In [4]:
ls /global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/

2d_4sigma-values/  3d_4sigma-values/  README.md  test_3d/


In [5]:
# data_dir='data_stored_results/results_3d_corr6_idx64_1000img/'
# data_dir='data_stored_results/results_3d_corr6_idx32_1000img/'

data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/'


In [24]:
flist=['3pt_corr_0_0.5.npy','3pt_corr_0_0.65.npy','3pt_corr_0_0.8.npy','3pt_corr_0_1.1.npy']
flist=glob.glob(data_dir+'*.npy')
flist+=glob.glob(data_dir+'*/*.npy')

In [7]:
# flist=['/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/3pt_corr_0_2d_val_0.8.npy']

In [6]:
flist=glob.glob(data_dir+'test_3d/img*.npy')
flist

['/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/test_3d/img_idx_70_3d_dset1.npy',
 '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/test_3d/img_idx_7_3d_dset1.npy',
 '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/test_3d/img_idx_0_3d_dset1.npy',
 '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/test_3d/img_idx_10_3d_dset1.npy',
 '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/test_3d/img_idx_100_3d_dset1.npy']

In [20]:
lst=[]
for i in [2,3,4]:
    arr=np.load(flist[i])
    lst.append(arr)
a1=np.stack(lst)


lst=[]
for i in [0,1]:
    arr=np.load(flist[i])
    lst.append(arr)
a2=np.stack(lst)

In [21]:
a1.shape,a2.shape

((3, 8, 8, 8), (2, 8, 8, 8))

In [15]:
cols=['fname','label','data','fle']
df=pd.DataFrame(columns=cols)

for count,fname in enumerate(flist):
    dict1={}
#     label=str(count)+'_'+str(fname.split('/')[-1].split('.npy')[0].split('corr')[-1])
    label=str(count)+'_'+str(fname.split('/')[-1].split('_')[-1].split('.npy')[0])
    sigma=fname.split('/')[-1].split('_')[-1].split('.npy')[0].split('-')[-1]
    val=np.load(fname)
    dict1={'label':label,'data':val,'sigma':sigma,'fle':fname.split('/')[-1],'fname':fname}
    df=df.append(dict1,ignore_index=True)


In [16]:
df.fle.values

array(['img_idx_70_3d_dset1.npy', 'img_idx_7_3d_dset1.npy',
       'img_idx_0_3d_dset1.npy', 'img_idx_10_3d_dset1.npy',
       'img_idx_100_3d_dset1.npy', '3pt_corr_0_1.1.npy',
       '3pt_corr_0_0.65.npy', '3pt_corr_0_0.8.npy', '3pt_corr_0_0.5.npy',
       '3pt_corr_0_1.1.npy', '3pt_corr_0_0.8.npy', '3pt_corr_0_0.5.npy'],
      dtype=object)

In [17]:
df[['label','fle','sigma']]

,label,fle,sigma
0,0_dset1,img_idx_70_3d_dset1.npy,dset1
1,1_dset1,img_idx_7_3d_dset1.npy,dset1
2,2_dset1,img_idx_0_3d_dset1.npy,dset1
3,3_dset1,img_idx_10_3d_dset1.npy,dset1
4,4_dset1,img_idx_100_3d_dset1.npy,dset1
5,5_1.1,3pt_corr_0_1.1.npy,1.1
6,6_0.65,3pt_corr_0_0.65.npy,0.65
7,7_0.8,3pt_corr_0_0.8.npy,0.8
8,8_0.5,3pt_corr_0_0.5.npy,0.5
9,9_1.1,3pt_corr_0_1.1.npy,1.1


## Plot fixed r1,r2

In [18]:
def f_get_corrs_fixed_r(arr,r1,r2):
    '''Get correlator values for given r1 and r2 '''    
    num_imgs=arr.shape[0]
    num_corrs=arr.shape[1]
    
    ### Pick r1,r2
    arr=arr[:,:,r1,r2].astype(float)
    # Avg over different images for fixed correlator
    mean=np.array([np.mean(arr[:,i]) for i in range(num_corrs)])
    err=np.array([np.std(arr[:,i])/np.sqrt(num_imgs) for i in range(num_corrs)])
    
    return mean,err

def f_plot_fixed_rs(corrs_arr_lst,labels_lst,r1,r2,mode='relative'):
    ''' mode =['absolute','relative']'''
    
    
    mean_ref,err_ref=f_get_corrs_fixed_r(corrs_arr_lst[0],r1,r2)
    
    plt.figure()
    for arr,label in zip(corrs_arr_lst,labels_lst):
        mean,err=f_get_corrs_fixed_r(arr,r1,r2)
        if mode=='relative':
            x,y,yerr=np.arange(len(mean)),mean/mean_ref,(mean/mean_ref)*np.sqrt((err/mean)**2 + (err_ref/mean_ref)**2)
        else : 
            x,y,yerr=np.arange(len(mean)),mean,err
        plt.errorbar(x,y,yerr,marker='o',label=label)
    #     plt.plot(x,y,marker='o',label=str(i))
        print(y/yerr)
    plt.legend()
    plt.xlabel('l')
    plt.ylabel('3ptfc')
    plt.title('3ptfc values for fixed {0}-{1} for different l values'.format(r1,r2))

def f_get_corrs_fixed_l(arr,l,r1):
    '''Get correlator values for given l and r1 '''    
    num_imgs=arr.shape[0]
    num_corrs=arr.shape[3]
    print(arr.shape)
    
    ### Pick l,r1
    arr=arr[:,l,r1,:].astype(float)

    # Avg over different images for fixed correlator
    mean=np.array([np.mean(arr[:,i]) for i in range(num_corrs)])
    err=np.array([np.std(arr[:,i])/np.sqrt(num_imgs) for i in range(num_corrs)])
    
    return mean,err


def f_plot_fixed_l(corrs_arr_lst,labels_lst,l,r1,mode='relative'):
    ''' mode =['absolute','relative']'''
    

    mean_ref,err_ref=f_get_corrs_fixed_l(corrs_arr_lst[0],l,r1)

    plt.figure()
    for arr,label in zip(corrs_arr_lst,labels_lst):
        mean,err=f_get_corrs_fixed_l(arr,l,r1)
        
        print(label,type(label))

        if mode=='relative':
            x,y,yerr=np.arange(len(mean)),mean/mean_ref,(mean/mean_ref)*np.sqrt((err/mean)**2 + (err_ref/mean_ref)**2)
        else : 
            x,y,yerr=np.arange(len(mean)),mean,err
        plt.errorbar(x,y,yerr,marker='o',label=label)
    #     plt.plot(x,y,marker='o',label=str(i))
#         print(y/yerr)

    plt.xticks(x)
    plt.xlabel('r2')
    plt.legend()
    plt.title('3ptfc values for fixed l={0} and r1: {1} for different r2 values'.format(l,r1))

# f_plot_fixed_l(corr_lst,['a1','a2'],l=3,r1=3,mode='relative')
# f_plot_fixed_rs(corr_lst,['a1','a2','a3'],r1=1,r2=4,mode='relative')


In [19]:

def f_plot_df_fixed_l(df,labels_lst,bkgnd_lst,l,r1,mode='relative'):
    ''' mode =['absolute','relative']'''
    
    mean_ref,err_ref=f_get_corrs_fixed_l(df.data.values[0],l,r1)
    
    plt.figure()
    
    df_temp=df[df.label.isin(labels_lst)]
    for i,row in df_temp.iterrows():
        arr=row.data
        label=row.label
        mean,err=f_get_corrs_fixed_l(arr,l,r1)

        if mode=='relative':
            x,y,yerr=np.arange(len(mean)),mean/mean_ref,(mean/mean_ref)*np.sqrt((err/mean)**2 + (err_ref/mean_ref)**2)
        else : 
            x,y,yerr=np.arange(len(mean)),mean,err
        plt.errorbar(x,y,yerr,marker='o',label=label)
    #     plt.plot(x,y,marker='o',label=str(i))
    
    df_temp=df[df.label.isin(bkgnd_lst)]
    for i,row in df_temp.iterrows():
        arr=row.data
        label=row.label
        mean,err=f_get_corrs_fixed_l(arr,l,r1)
        if mode=='relative':
            x,y,yerr=np.arange(len(mean)),mean/mean_ref,(mean/mean_ref)*np.sqrt((err/mean)**2 + (err_ref/mean_ref)**2)
        else : 
            x,y,yerr=np.arange(len(mean)),mean,err
        plt.errorbar(x,y,yerr,label=label,linestyle='--')
       
    plt.xticks(x)
    plt.xlabel('r2')
    plt.ylabel('3ptfc')
    plt.legend()
    plt.title('3ptfc values for fixed l={0} and r1: {1} for different r2 values'.format(l,r1))

def f_plot_df_fixed_rs(df,labels_lst,bkgnd_lst,r1,r2,mode='relative'):
    ''' mode =['absolute','relative']'''
    
    mean_ref,err_ref=f_get_corrs_fixed_r(df.data.values[0],r1,r2)
    
    plt.figure()
    
    df_temp=df[df.label.isin(labels_lst)]
    for i,row in df_temp.iterrows():
        arr=row.data
        label=row.label
        mean,err=f_get_corrs_fixed_r(arr,r1,r2)
        if mode=='relative':
            x,y,yerr=np.arange(len(mean)),mean/mean_ref,(mean/mean_ref)*np.sqrt((err/mean)**2 + (err_ref/mean_ref)**2)
        else : 
            x,y,yerr=np.arange(len(mean)),mean,err
        plt.errorbar(x,y,yerr,marker='o',label=label)
    #     plt.plot(x,y,marker='o',label=str(i))
    
    df_temp=df[df.label.isin(bkgnd_lst)]
    for i,row in df_temp.iterrows():
        arr=row.data
        label=row.label
        mean,err=f_get_corrs_fixed_r(arr,r1,r2)
        if mode=='relative':
            x,y,yerr=np.arange(len(mean)),mean/mean_ref,(mean/mean_ref)*np.sqrt((err/mean)**2 + (err_ref/mean_ref)**2)
        else : 
            x,y,yerr=np.arange(len(mean)),mean,err
        plt.errorbar(x,y,yerr,label=label,linestyle='--')
    
    plt.legend()
    plt.xlabel('l')
    plt.ylabel('3ptfc')
    plt.title('3ptfc values for fixed {0}-{1} for different l values'.format(r1,r2))
    
# f_plot_df_fixed_rs(df,labels_lst[:1],labels_lst,r1=1,r2=3,mode='absolute')
# f_plot_df_fixed_l(df,labels_lst[:1],labels_lst,l=1,r1=1,mode='absolute')

In [20]:
# # labels=['0__0_2d_val_0.5', '3__0_2d_val_0.5','7__0_2d_val_test0.5',
# #         '1__0_2d_val_1.1', '4__0_2d_val_1.1', '8__0_2d_pred1.1_13-31770',
# # #         '2__0_2d_pred0.65_14-34640', '6__0_2d_val_0.65', 
# # #          '5__0_2d_val_0.8',
# #        ]
# labels=df.label.values
# df_temp=df[df.label.isin(labels)]
# img_lst=df_temp.data.values.tolist()
# labels_lst=df_temp.label.values.tolist()
# # labels_lst=['a','b','c','d']
# print(labels_lst)
# f_plot_fixed_l(img_lst,labels_lst,l=3,r1=3,mode='absolute')

In [22]:
interact(f_plot_df_fixed_rs,df=fixed(df),
                labels_lst=SelectMultiple(options=df.label.values),
                bkgnd_lst=SelectMultiple(options=df.label.values),
                r1=IntSlider(min=0,max=19,step=1),r2=IntSlider(min=0,max=19,step=1),
                mode=['absolute'])


interactive(children=(SelectMultiple(description='labels_lst', options=('0_dset1', '1_dset1', '2_dset1', '3_ds…

<function __main__.f_plot_df_fixed_rs(df, labels_lst, bkgnd_lst, r1, r2, mode='relative')>

In [19]:
interact(f_plot_df_fixed_l,df=fixed(df),
                labels_lst=SelectMultiple(options=df.label.values),
                bkgnd_lst=SelectMultiple(options=df.label.values),
                l=ToggleButtons(options=np.arange(6)),
                r1=IntSlider(min=0,max=18,step=1),mode=['absolute'])


interactive(children=(SelectMultiple(description='labels_lst', options=('0_val-0.5', '1_val-0.65', '2_val-0.8'…

<function __main__.f_plot_df_fixed_l(df, labels_lst, bkgnd_lst, l, r1, mode='relative')>

In [31]:
# fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/img_9-corr_2d_val_0.8.npy'
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/3pt_corr_0_3d_dset1.npy'
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/img_idx_0_3d_dset1.npy'
a1=np.load(fname)
a1.shape

(8, 8, 8)

In [40]:
# a1.shape

In [32]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/val.npy'
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/2d_4sigma-values/3pt_corr_0_1.1.npy'
a1=np.load(fname,mmap_mode='r')


In [33]:
a1.shape

(90, 6, 18, 18)

In [11]:
a1[0,:].shape

(1, 64, 64, 64)